<a href="https://colab.research.google.com/github/aasiyasan/Training/blob/main/Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install langchain-openai
!pip install langchain_community
!pip install chromadb
!pip install langchain-google-genai
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 4.8 MB/s eta 0:00:00


In [ ]:
!pip install twilio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.2 MB/s eta 0:00:00


In [ ]:
# import libraries
import os
from langchain_community.document_loaders import PyPDFLoader
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

import sqlite3
from twilio.rest import Client
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from google.colab import userdata
import tweepy

In [ ]:
gemini_api_key = userdata.get("GoogleAPIKey")

In [ ]:
CHROMA_PATH = "Chroma"
# ----- Data Indexing Process -----
# load your pdf docs
DOC_PATH = "/content/Test.pdf"
# load your pdf doc
loader = PyPDFLoader(DOC_PATH)
pages = loader.load()
# split the doc into smaller chunks i.e. chunk_size=500
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(pages)

# get OpenAI Embedding model
#embeddings = HuggingFaceEmbeddings(model_name="nomic-ai/nomic-embed-text-v1", model_kwargs={'device': 'cpu'})
embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# embed the chunks as vectors and load them into the database.
db_chroma = Chroma.from_documents(chunks, embeddings, persist_directory=CHROMA_PATH)
# ----- Retrieval and Generation Process -----

# this is an example of a user question (query)
#query = 'what are the top risks mentioned in the document?'
query='Summarize Nike Fiscal 2025 results'


/tmp/ipython-input-502393349.py:14: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or da

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:

# retrieve context - top 5 most relevant (closests) chunks to the query vector
# (by default Langchain is using cosine distance metric)
docs_chroma = db_chroma.similarity_search_with_score(query, k=5)

# generate an answer based on given user query and retrieved context information
context_text = "\n\n".join([doc.page_content for doc, _score in docs_chroma])
# you can use a prompt template
PROMPT_TEMPLATE = """Answer the question based only on the following context:
{context}
Answer the question based on the above context: {question}.
Provide a detailed answer.
Don’t justify your answers.
Don’t give information not mentioned in the CONTEXT INFORMATION.
Do not say "according to the context" or "mentioned in the context" or similar.
"""

In [ ]:
# load retrieved context and user query in the prompt template
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query)

# call LLM model to generate the answer based on the given context and query
#using google genai instead of open AI here

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash" ,api_key=userdata.get('GoogleAPIKey'))

response_text = llm.predict(prompt)
print(response_text)

/tmp/ipython-input-3818356121.py:10: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response_text = llm.predict(prompt)


NIKE, Inc. reported its fiscal 2025 full year results ended May 31, 2025.

**Key Financial Highlights for Fiscal 2025:**
*   **Full year revenues:** $46.3 billion, down 10 percent on a reported basis compared to the prior year. On a currency-neutral basis, revenues were down 9 percent.
*   **NIKE Brand revenues:** $44.7 billion, down 9 percent on a reported and currency-neutral basis, driven by declines across all geographies.
*   **Net income:** $0.2 billion, down 86 percent.
*   **Diluted earnings per share:** $0.14, a decrease of 86 percent.
*   **Total NIKE, Inc. Earnings Before Interest and Taxes (EBIT):** $3,778 million for the twelve months ended 5/31/2025, a decrease of 42% compared to $6,539 million in the prior year.
*   **EBIT margin:** 8.2% for the full year 2025, compared to 12.7% in the prior year.
*   **Total NIKE, Inc. Income Before Income Taxes:** $3,885 million, a decrease of 42% compared to $6,700 million in the prior year.

**EBIT Performance by Segment (Twelve Mont

<All keys matched successfully>


# Stroing the Summary In SQLite

In [ ]:
conn = sqlite3.connect('responses.db')
cursor = conn.cursor()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS responses (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        query TEXT,
        response TEXT
    )
''')


cursor.execute('''
    INSERT INTO responses (query, response) VALUES (?, ?)
''', (query, response_text))


conn.commit()
conn.close()

# Twilo Credentials

In [ ]:
# Twilio credentials
account_sid = userdata.get("TwilioAccountId")
auth_token = userdata.get("TwiliouthToken")

# Sending the message to a WhatsAPP number

In [ ]:
twilio_whatsapp_number = userdata.get("Twilio_Whatsappsender")
recipient_number = userdata.get("WhatsappRecipient")

client = Client(account_sid, auth_token)
message = client.messages.create(
        body=response_text,
        from_=twilio_whatsapp_number,
        to=recipient_number
    )
print("Message sent! SID:", message.sid)


Message sent! SID: SM43c054453c993bf80c5ad8e090b3657c


# Sending the Summary as text message to the Phone no.

In [ ]:
twilio_phone_number = '+18777804236'
recipient_number = userdata.get("recipientphnumber")


client = Client(account_sid, auth_token)
message = client.messages.create(
        body=response_text,
        from_=twilio_phone_number,
        to=recipient_number
    )
print("SMS sent! SID:", message.sid)


Message sent successfully ! SID : Message sent! SID: SM44c056653e993bd10a5ce8d090b1237c


# Send the summary as an Email

In [ ]:

    # Email configuration
    sender_email = userdata.get("sender_email")
    sender_password = "uigkkansdpzlsqwq"
    recipient_email = userdata.get("recipient_email")
    subject = "Document Summary - Test"

    # Create the email message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = recipient_email
    msg['Subject'] = subject
    msg.attach(MIMEText(response_text, 'plain'))

    # 4. Send the email
    try:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, recipient_email, msg.as_string())
        print("Email sent successfully!")
    except Exception as e:
        print("Failed to send email:", e)

Email sent successfully!


# Posting the message in Twitter

In [ ]:
# Twitter API credentials (replace with your own)
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''


#Shorten to 280 characters if needed
if len(response_text) > 280:
    response_text = response_text[:277] + '...'
print("Tweet message:", response_text)

# Post to Twitter
auth = tweepy.OAuth1UserHandler(
    consumer_key, consumer_secret, access_token, access_token_secret
)
api = tweepy.API(auth)
try:
    api.update_status(response_text)
    print("Tweet posted successfully!")
except Exception as e:
    print("Failed to post tweet:", e)


Tweet message: NIKE, Inc. reported its fiscal 2025 full year results ended May 31, 2025.

**Key Financial Highlights for Fiscal 2025:**
*   **Full year revenues:** $46.3 billion, down 10 percent on a reported basis compared to the prior year. On a currency-neutral basis, revenues were down 9...
Failed to post tweet: 400 Bad Request
215 - Bad Authentication data.
